In [ ]:
# Import the general functions for metabolic modelling
%run 'general_functions.ipynb'

growth_norm = False # Calculated growth rates are normalized by the growth rate of that model instance 
p_diff = 0.15

In [ ]:
# Load one of thermodynamics models 
model = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_WT.xml")
subsystem_transfer(model)
tmodel = model
# Load the pTVA Results
tva_control = pickle.load(open("../Results/pTVA_Control_woBile.p","rb"))
tva_oxr = pickle.load(open("../Results/pTVA_OxR_woBile.p","rb"))
tva_dmso = pickle.load(open("../Results/pTVA_Control_DMSO_woBile.p","rb"))
tva_rur = pickle.load(open("../Results/pTVA_RuR_woBile.p","rb"))
fluxes = [tva_control,tva_oxr,tva_dmso,tva_rur]

In [ ]:
plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots(figsize=(20, 16))
width = 0.18
loc = [-width-width/2,-width/2,width/2,width+width/2]
cols = ["red","orange","black","gray"]
alphs = [0.5,0.5,1,1]
labels = ["OxR (before GRN)","RuR (before GRN)","OxR (after GRN)","RuR (after GRN)"]
counter = 0

for growth_norm in [False,True]:
    
    if growth_norm:
        for tva_fluxes in fluxes:
            normalize_growth(tva_fluxes,tmodel)
    
    for sensitive,resistant in [(tva_control,tva_oxr),(tva_dmso,tva_rur)]:
        altered_rxns = compare_fluxes(sensitive['minimum'],sensitive['maximum'],
                                      resistant['minimum'],resistant['maximum'],
                                      percent_difference=p_diff,zero_cut=1e-5)
        all_altered_rxns = [model.reactions[i] for i in altered_rxns]

        # Subsystem analysis 
        all_subsystems = set([r.subsystem for r in tmodel.reactions])
        all_ratios = []
        all_names = []
        all_altered_rxns = [model.reactions[i] for i in altered_rxns]
        for sub_names in list(sub_manual.values()):
            full_names = []
            # Get all corresponding subsystems 
            for sub_name in sub_names:
                full_name = [s for s in all_subsystems if sub_name in s]
                full_names = full_names + full_name
            # Get key for value from dictionary
            name = list(sub_manual.keys())[list(sub_manual.values()).index(sub_names)] 
            # Get reactions associated with that set of subsystems 
            sub_rxns = [r for r in model.reactions if r.subsystem in full_names]
            if len(sub_rxns) == 0:
                continue
            # Get reactions associated with set of subsystems and in altered_rxns 
            sub_altered = [r for r in sub_rxns if r in all_altered_rxns]
            sub_rxns_altered = [r for r in model.reactions if r in sub_altered]
            sub_ratio = len(sub_rxns_altered)/len(sub_rxns)
            if sub_ratio > 0.:
                all_ratios.append(sub_ratio)
                all_names.append(name)
        all_ratios = pd.Series(all_ratios)
        x = np.arange(len(all_names))
        ax.bar(x+loc[counter],all_ratios,width,label=labels[counter],color=cols[counter],alpha=alphs[counter])
        counter = counter + 1
        
ax.set_xticks(x)
ax.set_yticklabels([round(i,2) for i in np.arange(0,0.6,0.05)],rotation=90)
ax.set_xticklabels(all_names,rotation=90)
ax.legend()
fig.tight_layout()
fig.savefig("../Figures/Normalization/Barplot_Summary_woBile.svg".format(growth_norm))
plt.show()

In [ ]:

if growth_norm:
    
    fig, ax = plt.subplots(figsize=(20, 16))
    
    # Find the overall highest and lowest flux values in the results
    df_values = tva_control['minimum'] + tva_control['maximum'] + tva_oxr['minimum'] + tva_oxr['maximum'] \
                + tva_dmso['minimum'] + tva_dmso['maximum'] + tva_rur['minimum'] + tva_rur['maximum']
    lowest_flux = min(df_values)
    highest_flux = max(df_values)

    # Need to scale the zero cutoff value and the flux comparison cutoff values 
    scaled_zero_cut = feature_scaling([0.001],lowest_flux,highest_flux)[0] - feature_scaling([0],lowest_flux,highest_flux)[0]
    scaled_p_diff = feature_scaling([p_diff],lowest_flux,highest_flux)[0] - feature_scaling([0],lowest_flux,highest_flux)[0]
    p_diff = scaled_p_diff
    print(p_diff)
    
    # Apply feature scaling to the individual results!
    oxr_lower = feature_scaling(tva_oxr['minimum'],lowest_flux,highest_flux)
    oxr_upper = feature_scaling(tva_oxr['maximum'],lowest_flux,highest_flux)
    rur_lower = feature_scaling(tva_rur['minimum'],lowest_flux,highest_flux)
    rur_upper = feature_scaling(tva_rur['maximum'],lowest_flux,highest_flux)
    control_lower = feature_scaling(tva_control['minimum'],lowest_flux,highest_flux)
    control_upper = feature_scaling(tva_control['maximum'],lowest_flux,highest_flux)
    DMSO_lower = feature_scaling(tva_dmso['minimum'],lowest_flux,highest_flux)
    DMSO_upper = feature_scaling(tva_dmso['maximum'],lowest_flux,highest_flux)

    # Normalize the RuR and OxR results against their controls 
    # Check if model solution is then still feasible 
    rur_lower_norm = [i/j for i,j in zip(rur_lower,DMSO_lower)]
    rur_upper_norm = [i/j for i,j in zip(rur_upper,DMSO_upper)]
    oxr_lower_norm = [i/j for i,j in zip(oxr_lower,control_lower)]
    oxr_upper_norm = [i/j for i,j in zip(oxr_upper,control_upper)]
    
    altered_rxns = compare_fluxes(oxr_lower_norm,oxr_upper_norm,rur_lower_norm,rur_upper_norm,percent_difference=p_diff,zero_cut=scaled_zero_cut)
    all_altered_rxns = [tmodel.reactions[i] for i in altered_rxns]

    # Subsystem analysis 
    all_subsystems = set([r.subsystem for r in tmodel.reactions])
    all_ratios = []
    all_names = []
    all_altered_rxns = [model.reactions[i] for i in altered_rxns]
    for sub_names in list(sub_manual.values()):
        full_names = []
        # Get all corresponding subsystems 
        for sub_name in sub_names:
            full_name = [s for s in all_subsystems if sub_name in s]
            full_names = full_names + full_name
        # Get key for value from dictionary
        name = list(sub_manual.keys())[list(sub_manual.values()).index(sub_names)] 
        # Get reactions associated with that set of subsystems 
        sub_rxns = [r for r in model.reactions if r.subsystem in full_names]
        if len(sub_rxns) == 0:
            continue
        # Get reactions associated with set of subsystems and in altered_rxns 
        sub_altered = [r for r in sub_rxns if r in all_altered_rxns]
        sub_rxns_altered = [r for r in model.reactions if r in sub_altered]
        sub_ratio = len(sub_rxns_altered)/len(sub_rxns)
        if sub_ratio > 0. : #and "Nucleotide" not in name
            all_ratios.append(sub_ratio)
            all_names.append(name)
    all_ratios = pd.Series(all_ratios)
    x = np.arange(len(all_names))
    ax = all_ratios.plot(kind='bar',label="OxR vs RuR",color="black")
    counter = counter + 1
    ax.set_xticks(x)
    ax.set_yticklabels([round(i,2) for i in np.arange(0,0.6,0.05)],rotation=90)
    ax.set_xticklabels(all_names,rotation=90)
    ax.legend()
    fig.tight_layout()
    fig.savefig("../Figures/Normalization/Barplot_OxRvsRuR_woBile.svg".format(growth_norm))
    plt.show()
    

In [ ]:
if growth_norm:
    
    # Summary figure for the below results 
    sub_manual_oxr_rur = sub_manual

    rxns_count = dict()
    rxns_string = dict()
    rxns_cols = dict()
    all_subsystems = set([r.subsystem for r in tmodel.reactions])
    for sub in all_subsystems:
        print(" ")
        print(sub)
        sub_rxns = [r for r in tmodel.reactions if r.subsystem == sub]
        diff_rxns = list(set(sub_rxns).intersection(all_altered_rxns))
        rxns_names = []
        relative_fluxes = []
        rxns = []
        flux_diff = []
        for r in diff_rxns:
            i = tmodel.reactions.index(r)
            oxr_mean = (oxr_lower_norm[i]+oxr_upper_norm[i])/2
            rur_mean = (rur_lower_norm[i]+rur_upper_norm[i])/2
            relative_fluxes.append(relative_flux(rur_mean,oxr_mean))
            rxns_names.append(string_reaction(r))
            rxns.append(r)
            if oxr_mean > rur_mean:
                flux_diff.append("red")
                print("{} - higher in OxR".format(string_reaction(r)))
            else:
                flux_diff.append("orange")
                print("{} - higher in RuR".format(string_reaction(r)))
        if len(rxns) > 0: 
#             print(sub)
#             print(len(rxns))
#             print(" ")
            strings = ""
            for r in rxns:
                strings = strings + "\n" + string_reaction(r)
#                 print(string_reaction(r))
            col = max(set(flux_diff), key = flux_diff.count)
            rxns_cols[sub] = col
            rxns_count[sub] = len(rxns)
            rxns_string[sub] = strings


    # Transport reactions were ommited from the figure for clarity 
    labels = sub_manual_oxr_rur.keys()
    sizes = []
    cols = []
    sub_labels = []
    for sub in labels:
        if "Transport" not in sub:
            if "Exchange" not in sub:
#                 if "Nucleotide" not in sub:
#                     if "Lipid" not in sub:
                        names = sub_manual_oxr_rur[sub]
                        names = [n for n in rxns_count.keys() if any(na in n for na in names)]
                        sub_counter = sum([rxns_count[s] for s in names])
                        sub_strings = ''.join([rxns_string[s] for s in names])
                        sub_strings = sub + sub_strings
                        try: 
                            sub_cols = [rxns_cols[s] for s in names]
                            sub_cols = max(set(sub_cols), key = sub_cols.count)
                            sizes.append(sub_counter)
                            if "Lipid" in sub:
                                sub_cols = "orange"
                            cols.append(sub_cols)
                            sub_labels.append(sub)
                        except ValueError:
                            pass
    fig = plt.figure(1, figsize=(17,10))
    ax1 = fig.add_subplot(111)
    ax1.pie(sizes, labels=sub_labels,shadow=True,textprops={'fontsize': 18},colors=cols,autopct='%1.1f%%',explode=[0.05]*len(sizes))
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    #ax1.legend(labels,loc="lower left", bbox_to_anchor=(-1, -1))
    rpatch = mpatches.Patch(color='red', label='Higher flux in OxR')
    opatch = mpatches.Patch(color='orange', label='Higher flux in RuR')
    fig.legend(handles=[rpatch, opatch],loc="lower left",fontsize=18,bbox_to_anchor=(0.8,0.9))
    fig.tight_layout()
    fig.savefig("../Figures/Normalization/PieChart_Summary_woBile.svg")
    plt.show()